## Chapter 4: Graph Problems

### Unweighted Graphs

Below are the base classes `Edge` and `Graph` and the subclass `DiGraph` for directed graphs.
The `@dataclass` annotation just creates the `__init__()` method automatically that sets the variables defined in the class (must have type hint).

In [12]:
from dataclasses import dataclass
from typing import List

@dataclass
class Edge:
    u: int  # 'from' vertex index
    v: int  # 'to' vertex index
    
    def reversed(self):
        return Edge(self.v, self.u)
    
    def __str__(self):
        return f'{self.u} -> {self.v}'
    
    
# TODO: error handling for IndexOutOfBoundsError in vertex indices
class Graph(object):
    def __init__(self, vertices=[]):
        self._vertices = vertices  # vertex can be any type
        self._edges = [[] for _ in vertices]  # adjacency list
            
    @property
    def vertex_count(self) -> int:
        return len(self._vertices)

    @property
    def edge_count(self) -> int:
        return sum(map(len, self._edges))  # compact syntax reducing 2-way arrays

    def add_vertex(self, vertex) -> int:
        # does not check if vertex is already present (vertices are not hashed)
        self._vertices.append(vertex)
        self._edges.append([])
        return self.vertex_count - 1  # return index of added vertex

    def remove_vertex(self, vertex) -> None:
        # need to update all indices, expensive operation!
        # TODO: more efficient implementation?
        vertex_index = self.index_of(vertex)
        bad_edges = []
        for vtx in self._edges:
            for e in vtx:
                if vertex_index in [e.u, e.v] and\
                    e not in bad_edges and e.reversed() not in bad_edges:
                    bad_edges.append(e)
        [self.remove_edge(e) for e in bad_edges]
        for vtx in self._edges:
            for e in vtx:
                if e.u > vertex_index:
                    e.u -= 1
                if e.v > vertex_index:
                    e.v -= 1
        self._edges.pop(vertex_index)
        self._vertices.pop(vertex_index)
        
    def add_edge(self, edge: Edge) -> None:
        # in base case of undirected graph, add edges in both directions
        self._edges[edge.u].append(edge)
        self._edges[edge.v].append(edge.reversed())

    def add_edge_by_indices(self, u: int, v: int) -> None:
        # convenience method
        self.add_edge(Edge(u, v))

    def add_edge_by_vertices(self, first, second) -> None:
        # convenience method; will not work for duplicate vertex values!
        u: int = self._vertices.index(first)
        v: int = self._vertices.index(second)
        self.add_edge_by_indices(u, v)
        
    def remove_edge(self, edge: Edge) -> None:
        # in base case of undirected graph, remove edges in both directions
        self._edges[edge.u].remove(edge)
        self._edges[edge.v].remove(edge.reversed())
        
    def remove_edge_by_indices(self, u: int, v: int) -> None:
        # convenience method
        self.remove_edge(Edge(u, v))
        
    def remove_edge_by_vertices(self, first, second) -> None:
        # convenience method; will not work for duplicate vertex values!
        u: int = self._vertices.index(first)
        v: int = self._vertices.index(second)
        self.remove_edge_by_indices(u, v)

    def vertex_at(self, index: int):
        return self._vertices[index]

    def index_of(self, vertex) -> int:
        return self._vertices.index(vertex)

    def neighbors_for_index(self, index: int) -> List:
        return list(map(self.vertex_at, [e.v for e in self._edges[index]]))

    def neighbors_for_vertex(self, vertex) -> List:
        # convenience method
        # will provide the 'successors' function for seach algorithm!
        return self.neighbors_for_index(self.index_of(vertex))

    def edges_for_index(self, index: int) -> List[Edge]:
        return self._edges[index]

    def edges_for_vertex(self, vertex) -> List[Edge]:
        # convenience method
        return self.edges_for_index(self.index_of(vertex))

    def __str__(self):
        desc: str = ""
        for i in range(self.vertex_count):
            desc += f"{self.vertex_at(i)} -> {self.neighbors_for_index(i)}\n"
        return desc
    

# Directed unweighted graph
class DiGraph(Graph):
    def __init__(self, vertices=[]):
        super().__init__(vertices)
        
    def add_edge(self, edge: Edge) -> None:
        # overridden from base class
        self._edges[edge.u].append(edge)
        
    def remove_edge(self, edge: Edge) -> None:
        self._edges[edge.u].remove(edge)



Let's create a test dataset with American cities and their connections. Try it out as a regular undirected `Graph` or as a directed `DiGraph`.

In [2]:
city_graph = DiGraph(["Seattle", "San Francisco", "Los Angeles", "Riverside", "Phoenix", "Chicago", "Boston", "New York", "Atlanta", "Miami", "Dallas", "Houston", "Detroit", "Philadelphia", "Washington"])
city_graph.add_edge_by_vertices("Seattle", "Chicago")
city_graph.add_edge_by_vertices("Seattle", "San Francisco")
city_graph.add_edge_by_vertices("San Francisco", "Riverside")
city_graph.add_edge_by_vertices("San Francisco", "Los Angeles")
city_graph.add_edge_by_vertices("Los Angeles", "Riverside")
city_graph.add_edge_by_vertices("Los Angeles", "Phoenix")
city_graph.add_edge_by_vertices("Riverside", "Phoenix")
city_graph.add_edge_by_vertices("Riverside", "Chicago")
city_graph.add_edge_by_vertices("Phoenix", "Dallas")
city_graph.add_edge_by_vertices("Phoenix", "Houston")
city_graph.add_edge_by_vertices("Dallas", "Chicago")
city_graph.add_edge_by_vertices("Dallas", "Atlanta")
city_graph.add_edge_by_vertices("Dallas", "Houston")
city_graph.add_edge_by_vertices("Houston", "Atlanta")
city_graph.add_edge_by_vertices("Houston", "Miami")
city_graph.add_edge_by_vertices("Atlanta", "Chicago")
city_graph.add_edge_by_vertices("Atlanta", "Washington")
city_graph.add_edge_by_vertices("Atlanta", "Miami")
city_graph.add_edge_by_vertices("Miami", "Washington")
city_graph.add_edge_by_vertices("Chicago", "Detroit")
city_graph.add_edge_by_vertices("Detroit", "Boston")
city_graph.add_edge_by_vertices("Detroit", "Washington")
city_graph.add_edge_by_vertices("Detroit", "New York")
city_graph.add_edge_by_vertices("Boston", "New York")
city_graph.add_edge_by_vertices("New York", "Philadelphia")
city_graph.add_edge_by_vertices("Philadelphia", "Washington")

# city_graph.remove_vertex("Chicago")
print(city_graph)


Seattle -> ['Chicago', 'San Francisco']
San Francisco -> ['Riverside', 'Los Angeles']
Los Angeles -> ['Riverside', 'Phoenix']
Riverside -> ['Phoenix', 'Chicago']
Phoenix -> ['Dallas', 'Houston']
Chicago -> ['Detroit']
Boston -> ['New York']
New York -> ['Philadelphia']
Atlanta -> ['Chicago', 'Washington', 'Miami']
Miami -> ['Washington']
Dallas -> ['Chicago', 'Atlanta', 'Houston']
Houston -> ['Atlanta', 'Miami']
Detroit -> ['Boston', 'Washington', 'New York']
Philadelphia -> ['Washington']
Washington -> []



#### 4.3 Breadth-first search (BFS) for unweighted graphs

To find the shortest path in an unweighted graph, we'll be reusing the search algorithms originally created for solving Maze problems (Chapter 2). Notice how they can be reused on a different data structure (`Graph` vs `Maze`) with no changes to the code!

In [26]:
from typing import Deque
    

class Queue(object):
    def __init__(self):
        self._container = Deque()
        
    @property
    # creating property without setter forbids setting value
    def empty(self):
        return not self._container
    
    def push(self, item):
        self._container.append(item)
        
    def pop(self):
        return self._container.popleft()  # FIFO
    
    def __eq__(self, other):
        return self.__class__ == other.__class__ and\
                self._container == other._container
        
    def __repr__(self):
        return repr(self._container)


class Node(object):
    def __init__(self, state, parent):
        self.state = state
        self.parent = parent


class BFS(object):
    def __init__(self):
        self.count_states = 0
    
    def _init_frontier(self):
        return Queue()
    
    def _init_explored(self, initial):
        return {initial}
   
    def run(self, maze):
        return self._search(maze.start, maze.goal_test, maze.successors)
        
    def _search(self, initial, goal_test, successors):
        # 'goal_test' and 'successors' are callables!
        # frontier is where we've yet to go
        frontier = self._init_frontier()
        frontier.push(Node(initial, None))
        # explored is where we've been (implemented as set)
        explored = self._init_explored(initial)

        # keep going while there is more to explore
        while not frontier.empty:
            self.count_states += 1
            current_node = frontier.pop()
            current_state = current_node.state
            # if we found the goal, we're done
            if goal_test(current_state):
                return current_node
            # check where we can go next and haven't explored
            for child in successors(current_state):
                if child in explored:  # skip children we already explored
                    continue
                explored.add(child)
                frontier.push(Node(child, current_node))
        return None  # went through everything and never found goal

    def node_to_path(self, node):
        path = [node.state]
        # work backwards from end to front
        while node.parent is not None:
            node = node.parent
            path.append(node.state)
        path.reverse()
        return path

    
# Check how solution changes if data is a directed graph
bfs = BFS()
solution = bfs._search('Riverside', lambda x: x == 'Houston', lambda x: city_graph.neighbors_for_vertex(x))
if not solution:
    print('No solution found!\n')
else:
    print(f'Path length: {len(bfs.node_to_path(solution))}')
    print(f'Searched states: {bfs.count_states}\n')
    print(' -> '.join(bfs.node_to_path(solution)))
    

Path length: 3
Searched states: 5

Riverside -> Phoenix -> Houston


### Weighted Graphs

Now let's add a `weight` property to edges.

In [27]:
@dataclass
class WeightedEdge(Edge):
    # will inherit attributes of Edge
    weight: float

    def reversed(self):
        return WeightedEdge(self.v, self.u, self.weight)

    # so that we can order edges by weight to find the minimum weight edge
    def __lt__(self, other) -> bool:
        return self.weight < other.weight

    def __str__(self) -> str:
        return f'{self.u} {self.weight}> {self.v}'
    
    
class WeightedGraph(Graph):
    def __init__(self, vertices=[]):
        super().__init__(vertices)

    def add_edge_by_indices(self, u, v, weight) -> None:
        self.add_edge(WeightedEdge(u, v, weight))

    def add_edge_by_vertices(self, first, second, weight) -> None:
        u: int = self._vertices.index(first)
        v: int = self._vertices.index(second)
        self.add_edge_by_indices(u, v, weight)

    def neighbors_for_index_with_weights(self, index: int):
        distance_tuples = []
        for edge in self.edges_for_index(index):
            distance_tuples.append((self.vertex_at(edge.v), edge.weight))
        return distance_tuples

    def __str__(self):
        desc: str = ""
        for i in range(self.vertex_count):
            desc += f"{self.vertex_at(i)} -> {self.neighbors_for_index_with_weights(i)}\n"
        return desc
    

Let's use the same cities dataset with associated distances as weights.

In [28]:
city_graph2 = WeightedGraph(["Seattle", "San Francisco", "Los Angeles", "Riverside", "Phoenix", "Chicago", "Boston", "New York", "Atlanta", "Miami", "Dallas", "Houston", "Detroit", "Philadelphia", "Washington"])

city_graph2.add_edge_by_vertices("Seattle", "Chicago", 1737)
city_graph2.add_edge_by_vertices("Seattle", "San Francisco", 678)
city_graph2.add_edge_by_vertices("San Francisco", "Riverside", 386)
city_graph2.add_edge_by_vertices("San Francisco", "Los Angeles", 348)
city_graph2.add_edge_by_vertices("Los Angeles", "Riverside", 50)
city_graph2.add_edge_by_vertices("Los Angeles", "Phoenix", 357)
city_graph2.add_edge_by_vertices("Riverside", "Phoenix", 307)
city_graph2.add_edge_by_vertices("Riverside", "Chicago", 1704)
city_graph2.add_edge_by_vertices("Phoenix", "Dallas", 887)
city_graph2.add_edge_by_vertices("Phoenix", "Houston", 1015)
city_graph2.add_edge_by_vertices("Dallas", "Chicago", 805)
city_graph2.add_edge_by_vertices("Dallas", "Atlanta", 721)
city_graph2.add_edge_by_vertices("Dallas", "Houston", 225)
city_graph2.add_edge_by_vertices("Houston", "Atlanta", 702)
city_graph2.add_edge_by_vertices("Houston", "Miami", 968)
city_graph2.add_edge_by_vertices("Atlanta", "Chicago", 588)
city_graph2.add_edge_by_vertices("Atlanta", "Washington", 543)
city_graph2.add_edge_by_vertices("Atlanta", "Miami", 604)
city_graph2.add_edge_by_vertices("Miami", "Washington", 923)
city_graph2.add_edge_by_vertices("Chicago", "Detroit", 238)
city_graph2.add_edge_by_vertices("Detroit", "Boston", 613)
city_graph2.add_edge_by_vertices("Detroit", "Washington", 396)
city_graph2.add_edge_by_vertices("Detroit", "New York", 482)
city_graph2.add_edge_by_vertices("Boston", "New York", 190)
city_graph2.add_edge_by_vertices("New York", "Philadelphia", 81)
city_graph2.add_edge_by_vertices("Philadelphia", "Washington", 123)

# print(city_graph2)

#### 4.4.2 Minimum Spanning Tree (MST)

We will find the graph's MST using Jarnik's algorithm, also called Prim's algorithm:
* An MST exists because our graph is *connected*, *weighted* and *undirected*. If any of these does not hold, the algorithm may not find the MST.
* If the assumptions above hold, it does not matter from which vertex we start building the MST!
* The algorithm is *greedy*, so it always adds the lowest-weight edge to the tree first.
* To minimize the total edge weights in the tree, we will use a Priority Queue -- same as we did for the A* search in Maze problems (Chapter 2).

In [36]:
from heapq import heappush, heappop


class PriorityQueue(object):
    def __init__(self):
        self._container = []
        
    # creating property without setter forbids setting value
    @property
    def empty(self):
        return not self._container
    
    def push(self, item):
        heappush(self._container, item)  # in by priority
        
    def pop(self):
        return heappop(self._container)  # out by priority
    
    def __eq__(self, other):
        return self.__class__ == other.__class__ and\
                self._container == other._container
        
    def __repr__(self):
        return repr(self._container)

    
def mst(wg, start=0):
    if type(wg) != WeightedGraph:
        raise ValueError('Input graph must be a WeightedGraph')
    if start > (wg.vertex_count - 1) or start < 0:
        return None  # invalid start index
    result = [] # holds the final MST
    pq = PriorityQueue()
    visited = [False] * wg.vertex_count # where we've been

    def visit(index: int):
        visited[index] = True # mark as visited
        for edge in wg.edges_for_index(index):
            # add all edges coming from here to pq
            if not visited[edge.v]:
                pq.push(edge)

    visit(start) # the first vertex is where everything begins

    while not pq.empty: # keep going while there are edges to process
        edge = pq.pop()
        if visited[edge.v]:
            continue # don't ever revisit
        # this is the current smallest, so add it to solution
        result.append(edge)
        visit(edge.v) # visit where this connects
        
    return result



def total_weight(wp) -> float:
    return sum((edge.weight for edge in wp))


def print_weighted_path(wg, wp) -> None:
    for edge in wp:
        print(f"{wg.vertex_at(edge.u)} {edge.weight}> {wg.vertex_at(edge.v)}")
    print(f"Total Weight: {total_weight(wp)}")
    
    
    
# Find MST for weighted graph
result = mst(city_graph2)
if result is None:
    print("No solution found!")
else:
    print_weighted_path(city_graph2, result)
    
    

Seattle 678> San Francisco
San Francisco 348> Los Angeles
Los Angeles 50> Riverside
Riverside 307> Phoenix
Phoenix 887> Dallas
Dallas 225> Houston
Houston 702> Atlanta
Atlanta 543> Washington
Washington 123> Philadelphia
Philadelphia 81> New York
New York 190> Boston
Washington 396> Detroit
Detroit 238> Chicago
Atlanta 604> Miami
Total Weight: 5372


#### 4.5.1 Dijkstra's algorithm

In a weighted graph, Dijkstra's algorithm finds the shortest path _from a particular starting vertex_ to all other vertices of the graph.
* Unlike MST, finding the single-source shortest path depends on the starting vertex.
* Like Jarnik's MST algorithm, Dijkstra's algorithm is *greedy*, so it always explores the lowest-weight neighbor first.

In [31]:
@dataclass
class DijkstraNode:
    vertex: int
    distance: float

    def __lt__(self, other):
        return self.distance < other.distance

    def __eq__(self, other):
        return self.distance == other.distance


def dijkstra(wg, root):
    first = wg.index_of(root)  # find starting index
    # shortest distances are unknown at first
    distances = [None] * wg.vertex_count
    distances[first] = 0  # the root is 0 away from the root
    path_dict = {}  # how we got to each vertex
    pq = PriorityQueue()
    pq.push(DijkstraNode(first, 0))

    while not pq.empty:
        u = pq.pop().vertex  # explore the next closest vertex
        dist_u = distances[u]  # should already have seen it
        # look at every edge/vertex from the vertex in question
        for we in wg.edges_for_index(u):
            # the old distance to this vertex
            dist_v = distances[we.v]
            # no old distance or found shorter path
            if dist_v is None or dist_v > we.weight + dist_u:
                # update/overwrite distance to this vertex
                distances[we.v] = we.weight + dist_u
                # update the edge on the shortest path to this vertex
                path_dict[we.v] = we
                # explore it soon
                pq.push(DijkstraNode(we.v, we.weight + dist_u))

    return distances, path_dict


# Helper function to get easier access to dijkstra results
def distance_array_to_vertex_dict(wg, distances):
    distance_dict = {}
    for i in range(len(distances)):
        distance_dict[wg.vertex_at(i)] = distances[i]
    return distance_dict


# Takes a dictionary of edges to reach each node and returns a list of
# edges that goes from `start` to `end`
def path_dict_to_path(start, end, path_dict):
    if len(path_dict) == 0:
        return []
    edge_path  = []
    e = path_dict[end]
    edge_path.append(e)
    while e.u != start:
        e = path_dict[e.u]
        edge_path.append(e)
    return list(reversed(edge_path))



# Find shortest path
distances, path_dict = dijkstra(city_graph2, "Los Angeles")
name_distance = distance_array_to_vertex_dict(city_graph2, distances)
print("Shortest distances from Los Angeles:")
for key, value in name_distance.items():
    print(f"{key} : {value}")
print("")

print("Shortest path from Los Angeles to Boston:")
path = path_dict_to_path(city_graph2.index_of("Los Angeles"), city_graph2.index_of("Boston"), path_dict)
print_weighted_path(city_graph2, path)



Shortest distances from Los Angeles:
Seattle : 1026
San Francisco : 348
Los Angeles : 0
Riverside : 50
Phoenix : 357
Chicago : 1754
Boston : 2605
New York : 2474
Atlanta : 1965
Miami : 2340
Dallas : 1244
Houston : 1372
Detroit : 1992
Philadelphia : 2511
Washington : 2388

Shortest path from Los Angeles to Boston:
Los Angeles 50> Riverside
Riverside 1704> Chicago
Chicago 238> Detroit
Detroit 613> Boston
Total Weight: 2605


### Exercise: Bridges of Königsberg problem

The problem can be represented as an unweighted, undirected multigraph (multiple edges possible between two nodes):
* The four plots of land are 4 vertices 'A', 'B', 'C', 'D';
* The seven bridges joining the plots of land are 7 edges.

The algorithm for solving the problem should 'walk' the entire graph, visiting each edge _exactly_ once, and each vertex potentially more than once.

See more: https://en.wikipedia.org/wiki/Eulerian_path

In [41]:
bridges_graph = Graph(['A', 'B', 'C', 'D'])
bridges_graph.add_edge_by_vertices('A', 'C')
bridges_graph.add_edge_by_vertices('A', 'C')
bridges_graph.add_edge_by_vertices('A', 'D')
bridges_graph.add_edge_by_vertices('C', 'D')
bridges_graph.add_edge_by_vertices('C', 'B')
bridges_graph.add_edge_by_vertices('C', 'B')
bridges_graph.add_edge_by_vertices('B', 'D')

print(bridges_graph)


A -> ['C', 'C', 'D']
B -> ['C', 'C', 'D']
C -> ['A', 'A', 'D', 'B', 'B']
D -> ['A', 'C', 'B']



In [39]:
# Find MST for weighted graph
result = mst(bridges_graph)
if result is None:
    print("No solution found!")
else:
    print_weighted_path(bridges_graph, result)
    

A 1> D
D 1> C
D 1> B
Total Weight: 3
